In [1]:
from test.util import TestSuite
from src.util.goal import OneShotForecastingGoal

goal = OneShotForecastingGoal(view=5)
test_suite = TestSuite()
series1 = test_suite.__get_numpy_test_series_from_index__(6)

# Linear Regression

In [2]:

from src.performer.transformer import MinMaxTransformer
from src.util.evaluation import ForecastErrorEvaluation
from src.util.parameters import ModelsIperParameters
from src.validator.regressor.linear_regression import GridSearchKFoldCV_LinearRegression

LINEAR_REGRESSION_IPER_PARAMETERS = ModelsIperParameters(
    FEATURE_LENGTH=[5]
)
series = MinMaxTransformer.transform(series1)

ERROR_PERFORMER = ForecastErrorEvaluation(goal = goal)
forecast_view, forecast_offset = goal.options()

grid_searcher = GridSearchKFoldCV_LinearRegression(
    series = series,
    target_length = forecast_view,
    target_offset = forecast_offset,
    kfolds = 10
)
grid_searcher.search(LINEAR_REGRESSION_IPER_PARAMETERS, ERROR_PERFORMER)
loss, _, parameters, model = grid_searcher.get_best_params()

In [3]:
from src.performer.supervised_learning_performer import SlidingWindowPerformer
from src.performer.train_test_performer import TrainTestPerformer


SWP = SlidingWindowPerformer(
    feature_length=5,
    target_length=5,
    target_offset=1
)
TTP = TrainTestPerformer(portion_train = 0.8, random_sampling = True)

_, X, Y = SWP.get(series1)
_, X_test, _, Y_test = TTP.get(X, Y)
yhat_test = model.__test__(X_test)
error_eval = ForecastErrorEvaluation(goal=goal)
# yhat_test
# Y_test
error_eval.get(Y_test, yhat_test)

0.025187254

# CNN

In [4]:
from src.util.parameters import NeuralNetworkTrainingParameters
from src.validator.nn.trainer import GridSearch_MultiLayerPerceptronNetwork


MLP_IPER_PARAMETERS = ModelsIperParameters(
    FEATURE_LENGTH = [5]
)

series = MinMaxTransformer.transform(series)

ERROR_PERFORMER = ForecastErrorEvaluation(goal = goal)
forecast_view, forecast_offset = goal.options()

grid_searcher = GridSearch_MultiLayerPerceptronNetwork(
    series = series,
    target_length = forecast_view,
    target_offset = forecast_offset,
    training_parameters = NeuralNetworkTrainingParameters(EPOCHS = 900, EARLY_STOP = True, LEARNING_RATE = .001)
)

grid_searcher.search(MLP_IPER_PARAMETERS, ERROR_PERFORMER)
loss, parameters, model = grid_searcher.get_best_params()